In [6]:
import random
import os.path
import pandas as pd
import numpy as np
import csv

from pyspark.sql import SQLContext, Row
from pyspark.sql.types import *

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
# TODO: handle different types of NaN in different columns
# handle NAN for integers and check for '' or " "

In [8]:
def exists(filepath):
    return os.path.exists(filepath)

# Split 48 million rows to 10 million(training) & 38 million(testing)

In [11]:
if (exists('train10M.txt') and exists('test38M.txt')):
    print "train10M.txt and test38M.txt already created before."
else:
    #!split -l 2000000 train.txt ff
    #!cat ffaa ffat ffaf ffaq ffaj > train10M.txt
    #!rm ffaa ffat ffaf ffaq ffaj
    #!cat ff* > test38M.txt
    #!rm ff*
    print "Newly created files: train10M.txt and test38M.txt."

train10M.txt and test38M.txt already created before.


# Split training data into 3 partitions

In [12]:
if (exists('train5M.txt') and 
    exists('validation2M.txt') and 
    exists('test3M.txt')):
    print "train5M.txt, validation2M.txt, test3M.txt already created before."
else:
    !split -l 1000000 train10M.txt ff
    !cat ffaa ffaj ffad ffaf ffah > train5M.txt    
    !cat ffai ffae > validation2M.txt
    !cat ffac ffag ffab > test3M.txt
    !rm ff*
    print "Newly created files: train5M.txt, validation2M.txt, test3M.txt."

train5M.txt, validation2M.txt, test3M.txt already created before.


# Perform training and analysis with train5M data

In [21]:
df = pd.read_table('train5M.txt')
print df.shape

(4999999, 40)


# Remove and save the first column as y 

In [31]:
y = df['0']
y.to_csv('y_for_train5M.csv', index=False)
df = df.drop('0', 1)

# Rename column for train5m data

In [15]:
new_col_names = ['f' + str(num) for num in range(1,len(df.columns)+1)]
df.columns = new_col_names

# Get summary stats for all integer columns

In [16]:
all_columns = list(df.columns)
integer_cols = all_columns[:13]
df[integer_cols].describe()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13
count,2756813.000000,4999999.000000,3906401.000000,3853777.000000,4863896.000000,3887221.000000,4788996.000000,4997037.000000,4788996.000000,2756813.000000,4788996.000000,1177668.000000,3853777.000000
mean,3.550075,98.595417,30.839373,7.429061,18112.661813,118.854952,16.559919,12.785998,110.211554,0.610537,2.751485,0.946656,8.385096
std,9.865513,385.372407,494.089496,9.024720,68140.203950,421.572801,71.788627,19.683659,229.168662,0.690870,5.243586,5.745028,21.412443
min,0.000000,-3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,2.000000,2.000000,561.000000,8.000000,1.000000,2.000000,11.000000,0.000000,1.000000,0.000000,2.000000
50%,1.000000,2.000000,6.000000,4.000000,2802.000000,33.000000,3.000000,7.000000,40.000000,1.000000,1.000000,0.000000,4.000000
75%,3.000000,34.000000,17.000000,10.000000,10082.000000,104.000000,12.000000,19.000000,114.000000,1.000000,3.000000,0.000000,10.000000
max,1900.000000,257675.000000,65535.000000,563.000000,23159456.000000,367553.000000,56311.000000,5064.000000,29019.000000,9.000000,165.000000,1881.000000,7393.000000


# Create files for each histogram

In [ ]:
def m_count_distinct(col):
    """
        count distinct values in all columns
        @param col: numpy 1D array
        @return: dictionary of value, freq
    """
    d = {}
    for x in list(col):
        if type(x) == np.float64 and np.isnan(x):
            k = 'nan-' + col
            d[k] = d.get(k, 0) + 1
        else:
            d[x] = d.get(x, 0) + 1
    return d


# ==== create file from result of each freq count ===
for col in all_columns:
    filepath = 'histo-file/' + col + '.txt'
    if (exists(filepath)):
        print filepath + " already created."
    else:
        freq_count = m_count_distinct(df[col].values)
        with open(filepath, 'w') as f:
            for key, value in freq_count.items():
                row = str(key) + "," + str(value) + "\n"
                f.write(row)
        print "Newly created file:", filepath

# Create weighted distribution for each histogram and store in files

In [ ]:
for col in categorical_cols:
    filepath = "histo-weight/" + col + ".txt"
    if (exists(filepath)):
        print filepath + " already created."
    else:
        dfpath = "histo-file/" + col + ".txt"
        histo_df = pd.read_csv(dfpath, header=None, names=["val", "freq"])
        histo_df.freq = histo_df.freq / len(df)
        histo_df.columns = ["val", "weight"]
        histo_df.to_csv(filepath)
        print "Newly created file:" + filepath

# Plot histograms and store files

In [ ]:
for col in all_columns:
    filepath = "histo-file/" + col + ".txt"
    histo_df = pd.read_csv(filepath, header=None, names=["val", "freq"])
    ax = histo_df.plot(x='val', y='freq', logy='True', title=col, xlim=(0))
    fig = ax.get_figure()
    fig.savefig('second-histo-img/' + col + '.png')

# Normalize integer columns

In [17]:
df_norm = df[integer_cols].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
df_norm.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13
0,0.001053,0.000012,0.000671,0.001776,4.404249e-06,0.000022,0.000036,0.000395,0.000138,0.111111,0.006061,NaN,0.000541
1,0.001053,0.000012,0.000015,0.024867,3.311822e-05,0.000242,0.000071,0.000395,0.008443,0.111111,0.018182,0.001595,0.006087
2,NaN,0.003477,NaN,NaN,1.896418e-04,NaN,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN
3,0.001579,0.000008,NaN,0.000000,8.635781e-08,0.000000,0.000053,0.000000,0.000000,0.111111,0.006061,NaN,0.000000
4,NaN,0.000008,NaN,NaN,5.537263e-04,NaN,0.000000,0.000000,0.000207,NaN,0.000000,NaN,NaN


# Copy categorical columns into normalized dataframe

In [ ]:
categorical_cols = all_columns[13:]
df_norm[categorical_cols] = df[categorical_cols]
df_norm.head()

# Export normalized dataframe to file

In [ ]:
df_norm.to_csv("normalized_full_df.csv", index=False)

# Compute one-hot weighted distribution of features

In [15]:
df_onehot = pd.read_csv('normalized_full_df.csv')
df_onehot.head()

,Unnamed: 0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39
0,0,0.001053,0.000012,0.000671,0.001776,4.404249e-06,0.000022,0.000036,0.000395,0.000138,...,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,NaN,3a171ecb,43f13e8b,e8b83407,731c3655
1,1,0.001053,0.000012,0.000015,0.024867,3.311822e-05,0.000242,0.000071,0.000395,0.008443,...,8efede7f,3412118d,NaN,NaN,e587c466,ad3062eb,3a171ecb,3b183c5c,NaN,NaN
2,2,NaN,0.003477,NaN,NaN,1.896418e-04,NaN,0.000000,0.000000,0.000000,...,1e88c74f,74ef3502,NaN,NaN,6b3a5ca6,NaN,3a171ecb,9117a34a,NaN,NaN
3,3,0.001579,0.000008,NaN,0.000000,8.635781e-08,0.000000,0.000053,0.000000,0.000000,...,1e88c74f,26b3c7a7,NaN,NaN,21c9516a,NaN,32c7478e,b34f3128,NaN,NaN
4,4,NaN,0.000008,NaN,NaN,5.537263e-04,NaN,0.000000,0.000000,0.000207,...,776ce399,92555263,NaN,NaN,242bb710,8ec974f4,be7c41b4,72c78f11,NaN,NaN


### create a giant dictionary containing all categorical features and their weights

In [16]:
d = {}
categorical_cols = ['f' + str(num) for num in range(14,40)]
for col in categorical_cols:
    filepath = "histo-weight/" + col + ".txt"
    # print filepath
    with open(filepath) as f:
        reader = csv.reader(f)
        for ignore, val, weight in reader:
            d[val] = weight
print len(d)

5312649


### change every hashed string to their weighted values

In [ ]:
for col in categorical_cols:
    df_onehot[col] = df_onehot[col].apply(lambda x: d.get(x))
df_onehot.head()

### Export weighted onehot encoded dataframe to file

In [26]:
df_onehot.to_csv('one-hot-ready.csv', index=False)

# Select features to be used

In [ ]:
cols_to_keep = all_columns[10:13]
#df_norm[cols_to_keep].head()

# Model and Evaluation

In [51]:
#X = pd.read_csv('one-hot-ready.csv')
print X.shape
X.head()

(4999999, 39)


,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39
0,0.001053,0.000012,0.000671,0.001776,4.404249e-06,0.000022,0.000036,0.000395,0.000138,0.111111,...,0.129438,0.007669,0.332528,0.180496,5.692601e-03,NaN,0.203219,0.005693,0.114505,0.005693
1,0.001053,0.000012,0.000015,0.024867,3.311822e-05,0.000242,0.000071,0.000395,0.008443,0.111111,...,0.042881,0.000741,NaN,NaN,1.182160e-02,0.136544,0.203219,0.047778,NaN,NaN
2,NaN,0.003477,NaN,NaN,1.896418e-04,NaN,0.000000,0.000000,0.000000,NaN,...,0.041832,0.005630,NaN,NaN,2.000000e-07,NaN,0.203219,0.018353,NaN,NaN
3,0.001579,0.000008,NaN,0.000000,8.635781e-08,0.000000,0.000053,0.000000,0.000000,0.111111,...,0.041832,0.000500,NaN,NaN,5.163001e-03,NaN,0.434402,0.046772,NaN,NaN
4,NaN,0.000008,NaN,NaN,5.537263e-04,NaN,0.000000,0.000000,0.000207,NaN,...,0.052262,0.001148,NaN,NaN,8.944002e-04,0.006118,0.054689,0.001206,NaN,NaN


In [80]:
y = pd.read_csv('y_for_train5M.csv', header=None, names=["predict"])
print "shape:", y.shape
print y.head()

shape: (4999999, 1)
   predict
0        0
1        0
2        0
3        0
4        0


### Handle NaN

In [81]:
cX = X.fillna(0)
cy = y.fillna(0)

### convert y to 1D array

In [82]:
cy = cy.predict.values

### Logistic Regression

In [89]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(cX, cy)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [91]:
model.score(cX,cy)

0.74802934960586998

In [90]:
########### ================ STOPPP: still editing code below ####################

### Split up the data set into test and training sets

In [ ]:
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
testLogRModel = LogisticRegression()
testLogRModel.fit(x_train, y_train)

In [ ]:
testLogRModel.score(X,y)

### Predict class labels for the test set

In [ ]:
predicted = testLogRModel.predict(x_test)
print predicted

### AUC score from ROC

In [ ]:
from sklearn import metrics
print "accuracy:", metrics.accuracy_score(y_test, predicted)
# print "auc:", metrics.roc_auc_score(y_test, predicted)

### Perform cross validation using n folds

In [ ]:
from sklearn.cross_validation import cross_val_score
#n = 5
#scores = cross_val_score(LogisticRegression(), X, y, scoring='accuracy')
#print scores
#print scores.mean()

### Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf_random_forest = RandomForestClassifier(n_estimators=10)
clf_random_forest.fit(X,y)

In [ ]:
clf_random_forest.score(X,y)